# BigMac Dashboard

### Imports

In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

In [5]:
bigmac_data = pd.read_csv("data/raw/bigmac_price.csv")
wage_data = pd.read_csv("data/raw/wage.csv")

# Data Wrangling
bigmac_data['year'] = pd.to_datetime(bigmac_data['date']).dt.year
wage_data['year'] = wage_data['TIME'].astype(int)
bigmac_data_unique = bigmac_data.drop_duplicates(subset=['name', 'year'])
wage_data_simplified = wage_data[['Country', 'year', 'Value']].rename(columns={'Value': 'annual_wage_usd_ppp'}).drop_duplicates(subset=['Country', 'year'])
wage_data_simplified['hourly_wage_usd_ppp'] = wage_data_simplified['annual_wage_usd_ppp'] / (40 * 52)
final_data_cleaned = pd.merge(bigmac_data_unique, wage_data_simplified, left_on=['name', 'year'], right_on=['Country', 'year'], how='inner').drop_duplicates(subset=['name', 'year']).dropna()
final_data_cleaned['big_macs_per_hour'] = final_data_cleaned['hourly_wage_usd_ppp'] / final_data_cleaned['dollar_price']


In [6]:
# Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Big Mac Index Dashboard"),
    dcc.Dropdown(
        id='country-selector',
        options=[{'label': i, 'value': i} for i in final_data_cleaned['name'].unique()],
        value='United States'  # Default value
    ),
    dcc.Graph(id='big-mac-index-plot'),
])

@app.callback(
    Output('big-mac-index-plot', 'figure'),
    [Input('country-selector', 'value')]
)
def update_graph(selected_country):
    filtered_data = final_data_cleaned[final_data_cleaned['name'] == selected_country]
    fig = px.line(filtered_data, x='year', y='big_macs_per_hour', title=f"Big Macs per Hour in {selected_country}")
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)